## Shows all edges on screen

In [1]:
import cv2
import numpy as np

something = 3

# Load the video file
# cap = cv2.VideoCapture('./Dataset/Test/Subject1.MOV')  # Replace 'cricket_pitch_video.mp4' with your video file path
cap = cv2.VideoCapture('./Dataset/Alldone/18_Kan_C.mp4')  # Replace 'cricket_pitch_video.mp4' with your video file path

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        # If the video reaches the end, reset it to the beginning
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('Gray Scale',gray)

    # # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (something, something), 0)
    # cv2.imshow('blurred',blurred)

    # # Perform Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)
    cv2.imshow('blurred',edges)

    # # Perform line detection using HoughLinesP
    # lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

    # # Draw the detected lines on the frame
    # if lines is not None:
    #     for line in lines:
    #         x1, y1, x2, y2 = line[0]
    #         cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # # Display the frame with the detected crease lines
    # cv2.imshow('Crease Lines Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Detects white and batsman

In [2]:
import cv2
import mediapipe as mp
# import cvzone
# from cvzone.FaceMeshModule import FaceMeshDetector

# Open the video file
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
cap = cv2.VideoCapture('./Dataset/Alldone/18_Kan_C.mp4')  # Replace 'video.mp4' with the path to your video file

# Read the video frames
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # FACE
        # mp_drawing.draw_landmarks(frame, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_CONTOURS)
        # RIGHT HAND
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # LEFT HAND
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # POSE
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply adaptive thresholding to segment white regions
        _, threshold = cv2.threshold(gray_frame, 200, 255, cv2.THRESH_BINARY)

        # Find contours in the thresholded frame
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Iterate over the contours and draw bounding rectangles around white lines
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display the frame with bounding boxes
        cv2.imshow('Video', frame)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


## Removes everything except White

In [3]:
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture('./Dataset/Alldone/18_Kan_C.mp4')  # Replace 'video.mp4' with the path to your video file

# Read the video frames
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper white color thresholds (in HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])

    # Create a mask for the white color range
    mask = cv2.inRange(hsv_frame, lower_white, upper_white)

    # Apply the mask to the frame to detect white regions
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # Display the frame with white regions detected
    cv2.imshow('Video', result)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


## 2D Image output